In [45]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [46]:
path_evaluations_dir = os.path.join('..', '..', 'evaluations', 'mAP', 'mAP_2024-05-30-09-53-30')
path_evaluations_pkl = os.path.join(path_evaluations_dir, 'mAP_results_dict.pkl')
df = pd.read_pickle(path_evaluations_pkl)
df.keys() ## each key is the path to a different model .pth from the training runs

dict_keys(['..\\..\\models\\reid\\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-05-20-04-33__0__densenet121__PT-True\\best_densenet121__PT-True.pth', '..\\..\\models\\reid\\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-05-20-04-33__1__densenet121__PT-True\\best_densenet121__PT-True.pth', '..\\..\\models\\reid\\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-06-02-00-08__0__densenet121__PT-True\\best_densenet121__PT-True.pth', '..\\..\\models\\reid\\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-06-02-00-08__1__densenet121__PT-True\\best_densenet121__PT-True.pth', '..\\..\\models\\reid\\Other_species_sea-star-re-id-full_event-unaware_training-split_2024-05-03-18-13__2024-05-06-07-11-15__0__densenet121__PT-True\\best_densenet121__PT-True.pth', '..\\..\\models\\reid\\Other_species_sea-star-re-id-full_event-unaware_training

In [47]:
def get_numeric_info_from_settings(file_path, key):
    """
    Get the numeric value of a key from a settings file.
    example file contents
    Pretrained: True
    Model Version: densenet121
    Dataset: Reduced_Pycnopodia_helianthoides_full_event-unaware_training-split_2024-05-03-20-24
    Embedding Size: 256
    Number of Layers: 3
    Batch Size: 8
    Image Size: 256x256
    Margin: 1.0
    Learning Rate: 0.001
    Number of Epochs: 50
    Unfreeze Epoch: 51
    Unfreeze All Epoch: 1
    Number of Unfreeze Layers: 128
    Optimizer: sgd
    Momentum: 0.9
    Scheduler Step Size: 1
    Scheduler Gamma: 0.5
    Number of Workers: 4
    Prefetch Factor: 2

    :param file_path:
    :param key:
    :return:
    """


    with open(file_path, 'r') as file:
        for line in file:
            if key in line:
                # Split the line into key and value parts
                key, value = line.split(':')
                # Return the value part as an integer
                return float(value.strip())
    return None

In [48]:
path_models = sorted(list(df.keys()))

path_datasets = []
mAP_scores = []
embeddings = []
path_settings_files = []
margins = []
batch_sizes = []
num_epochs = []
embedding_dim = []
n_layers = []
unfreeze_layers_epoch = []
unfreeze_all_epoch = []
n_unfreeze_layers = []

pbar = tqdm(total=len(path_models))
for path_model in path_models[:]:
    tmp = df[path_model]
    path_datasets.append(tmp[1])
    mAP_scores.append(tmp[2])
    path_embedding_pkl = tmp[3]
    em = pd.read_pickle(path_embedding_pkl)
    embeddings.append(em)

    path_settings_file = os.path.join(os.path.dirname(path_model), 'logs', 'settings.txt')
    path_settings_files.append(path_settings_file)

    margins.append(get_numeric_info_from_settings(path_settings_file, 'Margin'))
    batch_sizes.append(get_numeric_info_from_settings(path_settings_file, 'Batch Size'))
    num_epochs.append(get_numeric_info_from_settings(path_settings_file, 'Number of Epochs'))
    embedding_dim.append(get_numeric_info_from_settings(path_settings_file, 'Embedding Size'))
    n_layers.append(get_numeric_info_from_settings(path_settings_file, 'Number of Layers'))
    unfreeze_layers_epoch.append(get_numeric_info_from_settings(path_settings_file, 'Unfreeze Epoch'))
    unfreeze_all_epoch.append(get_numeric_info_from_settings(path_settings_file, 'Unfreeze All Epoch'))
    n_unfreeze_layers.append(get_numeric_info_from_settings(path_settings_file, 'Number of Unfreeze Layers'))

    pbar.update(1)
pbar.close()


## combine all the data into a dataframe
df_data = pd.DataFrame({'path_model': path_models,
                        'path_dataset': path_datasets,
                        'mAP_score': mAP_scores,
                        'embedding': embeddings,
                        'path_settings_file': path_settings_files,
                        'margin': margins,
                        'batch_size': batch_sizes,
                        'num_epochs': num_epochs,
                        'embedding_dim': embedding_dim,
                        'n_layers': n_layers,
                        'unfreeze_layers_epoch': unfreeze_layers_epoch,
                        'unfreeze_all_epoch': unfreeze_all_epoch,
                        'n_unfreeze_layers': n_unfreeze_layers})
df_data.head()




100%|██████████| 347/347 [00:24<00:00, 14.31it/s]


,path_model,path_dataset,mAP_score,embedding,path_settings_file,margin,batch_size,num_epochs,embedding_dim,n_layers,unfreeze_layers_epoch,unfreeze_all_epoch,n_unfreeze_layers
0,..\..\models\reid\Other_species_sea-star-re-id...,..\..\data\Other_species_sea-star-re-id-full_e...,"{1: 0.8, 5: 0.9418604651162791, 10: 0.97209302...",i...,..\..\models\reid\Other_species_sea-star-re-id...,0.3,8.0,5.0,64.0,1.0,15.0,25.0,128.0
1,..\..\models\reid\Other_species_sea-star-re-id...,..\..\data\Other_species_sea-star-re-id-full_e...,"{1: 0.8093023255813954, 5: 0.9395348837209302,...",i...,..\..\models\reid\Other_species_sea-star-re-id...,0.3,8.0,5.0,64.0,1.0,15.0,25.0,128.0
2,..\..\models\reid\Other_species_sea-star-re-id...,..\..\data\Other_species_sea-star-re-id-full_e...,"{1: 0.7604651162790698, 5: 0.8976744186046511,...",i...,..\..\models\reid\Other_species_sea-star-re-id...,0.3,8.0,5.0,64.0,2.0,15.0,25.0,128.0
3,..\..\models\reid\Other_species_sea-star-re-id...,..\..\data\Other_species_sea-star-re-id-full_e...,"{1: 0.6930232558139535, 5: 0.8697674418604651,...",i...,..\..\models\reid\Other_species_sea-star-re-id...,0.3,8.0,5.0,64.0,2.0,15.0,25.0,128.0
4,..\..\models\reid\Other_species_sea-star-re-id...,..\..\data\Other_species_sea-star-re-id-full_e...,"{1: 0.7186046511627907, 5: 0.8813953488372093,...",i...,..\..\models\reid\Other_species_sea-star-re-id...,0.3,8.0,5.0,64.0,3.0,15.0,25.0,128.0


In [49]:
## save the assembled data into a pickle file inside of the evaluations directory
path_data_pkl = os.path.join(path_evaluations_dir, 'mAP_data.pkl')
df_data.to_pickle(path_data_pkl)